In [ ]:
####    新建一个笔记本文件（我把它命名为ch6_label_twitter），指定数据集的名称
import os
input_filename = os.path.join(os.path.expanduser("~"), "Data", "twitter", "python_tweets.json")
labels_filename = os.path.join(os.path.expanduser("~"), "Data", "twitter", "python_classes.json")
import json
tweets = []

with open(input_filename) as inf:
    for line in inf:
        if len(line.strip()) == 0: #跳过空行
            continue
        tweets.append(json.loads(line))

'''
    我们要实现以下功能，向用户（你）展示一条消息，要求用户输入类别：相关还是不相关。程序保存输入结果，继续展示下一条待标注的消息。
    首先，创建用于存储类别（标注结果）的列表。不管消息是不是与编程语言Python相关，我们都保存它的类别，
分类器从这两类数据中学习如何预测一条消息的类别。
    我们还要检测是不是有部分消息已经标注过类别了，有的话就加载这些类别。如果你标注到一半，临时有事要关闭笔记本文件，有了该功能，再打开时，
代码就会加载已有类别。一般来说，对于类似的任务，考虑如何保存中间结果很有必要。这样即使计算机中途死机，努力一个小时得
来的工作成果也不至于全部丢失！代码如下：'''
labels = []
if os.path.exists(labels_filename):
    with open(labels_filename) as inf:
        labels = json.load(inf)

'''
接下来，创建一个简单的函数，用来返回下一条需要标注的消息。我们找到并返回第一条没有标注类别的消息即可。代码如下。
'''        
def get_next_tweet():
    return tweet_sample[len(labels)]['text']

In [ ]:
'''
在笔记本中创建JavaScript程序来收集输入。可以借助魔术方法（magic function）在笔记本中直接嵌入HTML和JavaScript代码等。在笔记本的新格子中输入
'''
%%javascript
function set_label(label){
    var kernel = IPython.notebook.kernel;
    kernel.execute("labels.append(" + label + ")");
    load_next_tweet();
}
function load_next_tweet(){
    var code_input = "get_next_tweet()";
    var kernel = IPython.notebook.kernel;
    var callbacks = { 'iopub' : {'output' : handle_output}};
    kernel.execute(code_input, callbacks, {silent:false});
}
function handle_output(out){
    var res = out.content.data["text/plain"];
    $("div#tweet_text").html(res);
}


'''
接着，我们来使用另一个魔术方法%%html。毫无疑问，它是用来直接在笔记本中嵌入HTML代码。在新格子中输入如下代码。
'''
%%html
<div name="tweetbox">
    Instructions: Click in textbox. Enter a 1 if the tweet is relevant, enter 0 otherwise.<br>
Tweet: <div id="tweet_text" value="text"></div><br>
<input type=text id="capture"></input><br>
</div>
'''
先不要运行这个格子！
'''

<script>
$("input#capture").keypress(function(e) {
if(e.which == 48) {
    set_label(0);
    $("input#capture").val("");
}else if (e.which == 49){
    set_label(1);
    $("input#capture").val("");
  }
});

load_next_tweet();
</script>


'''
完成标注后，把所有的类别信息输出到前面定义好的类别文件中。
'''
with open(labels_filename, 'w') as outf:
json.dump(labels, outf)